In [6]:
from neo4j import GraphDatabase
from ollama import chat

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

driver = GraphDatabase.driver(URI, auth=AUTH)

def run_query(cypher_query, parameters=None):
    with driver.session() as session:
        result = session.run(cypher_query, parameters or {})
        return [record.data() for record in result]


# 查询含关键词的所有职业（模糊搜索）
query_fuzzy_jobs = """
MATCH (j:Job)
WHERE toLower(j.name) CONTAINS toLower($partial)
RETURN DISTINCT j.name AS job_title
"""

# 推荐课程 + 覆盖技能数量
query_recommend_courses_with_coverage = """
MATCH (j:Job {name: $job_title})-[:REQUIRES]->(s:Skill)
WITH COLLECT(s.name) AS required_skills
MATCH (c:Course)-[:TEACHES]->(s:Skill)
WHERE s.name IN required_skills
WITH c.name AS course, COUNT(DISTINCT s) AS coverage
RETURN course, coverage
ORDER BY coverage DESC
"""

# 技能缺口
query_missing_skills = """
MATCH (j:Job {name: $job_title})-[:REQUIRES]->(s:Skill)
WHERE NOT EXISTS {
  MATCH (:Course)-[:TEACHES]->(s)
}
RETURN DISTINCT s.name AS skill
"""

# 获取课程教授的技能
query_course_skills = """
MATCH (c:Course {name: $course_name})-[:TEACHES]->(s:Skill)
RETURN DISTINCT s.name AS skill
"""

# === 主流程 ===
if __name__ == "__main__":
    print("🔍 Keyword search: Enter part of a job name (e.g., 'data', 'java'):")
    keyword = input("Keyword: ").strip()

    matches = run_query(query_fuzzy_jobs, {"partial": keyword})
    if not matches:
        print("❌ No matching jobs found.")
        exit()

    print("\n✅ Matching jobs:")
    for i, row in enumerate(matches, 1):
        print(f"{i}. {row['job_title']}")

    index = int(input("\nSelect the job number you want to explore: "))
    job_title = matches[index - 1]["job_title"]

    print(f"\n🎯 Target job: {job_title}")

    # 推荐课程及其覆盖度
    recommended = run_query(query_recommend_courses_with_coverage, {"job_title": job_title})
    if recommended:
        print("\n📘 Recommended Courses (sorted by skill coverage):")
        for i, row in enumerate(recommended, 1):
            print(f"{i}. {row['course']}  —  covers {row['coverage']} required skill(s)")
    else:
        print("❌ No course recommendations found.")

    # 技能缺口
    missing_skills = run_query(query_missing_skills, {"job_title": job_title})
    if missing_skills:
        print("\n⚠️  Skills required but not taught:")
        for i, row in enumerate(missing_skills, 1):
            print(f"{i}. {row['skill']}")

    # 摘要生成
    choice = input("\nWould you like a course summary? (y/n): ").lower()
    while choice == "y":
        course_name = input("Enter the course name: ").strip()
        course_list = [r["course"] for r in recommended]
        if course_name in course_list:
            skills = run_query(query_course_skills, {"course_name": course_name})
            prompt = f"Summarize the course '{course_name}' for a student preparing to become a {job_title}. It teaches the following skills: "
            prompt += ", ".join([s['skill'] for s in skills])
            print("\n🤖 LLaMA3 Summary:\n")
            stream = chat(
                model="llama3",
                messages=[{"role": "user", "content": prompt}],
                stream=True,
            )
            for chunk in stream:
                print(chunk['message']['content'], end='', flush=True)
            print()
        else:
            print("❌ This course is not in the recommended list.")
        choice = input("\nAnother course summary? (y/n): ").lower()

    print("\n✅ Done. Thank you for using the smart recommender!")


🔍 Keyword search: Enter part of a job name (e.g., 'data', 'java'):

✅ Matching jobs:
1. Java Developer

🎯 Target job: Java Developer

📘 Recommended Courses (sorted by skill coverage):
1. Introduction to Computer Science  —  covers 1 required skill(s)
2. Lab: Basics of Programming  —  covers 1 required skill(s)
3. Software Engineering for Business Applications - Master Course: Web Application Engineering  —  covers 1 required skill(s)

⚠️  Skills required but not taught:
1. angular
2. typescript
❌ This course is not in the recommended list.

✅ Done. Thank you for using the smart recommender!
